Twitter Scraping

In [ ]:
!pip3 install snscrape

In [ ]:
import pandas as pd
import snscrape.modules.twitter as sntwitter

In [ ]:
tweets = []
topics = ['#OpenAI',"#SARS-CoV-2","#Chandrayaan3","#INDvAUS","#LPGPriceHike","#BodhDiwas_Of_SantGaribdasJi","#MPBudget2023","#MadeForTrade","#Messi","AI"]
for i in topics:
  scraper = sntwitter.TwitterSearchScraper(i)
  c = 0
  for tweet in scraper.get_items():
    if c>=1000:
      break;
    data=[i,tweet.content]
    tweets.append(data)
    c= c+1

<ipython-input-3-b2854c020c48>:9: FutureWarning: content is deprecated, use rawContent instead
  data=[i,tweet.content]


In [ ]:
tweet_df = pd.DataFrame(tweets,columns=["topic","content"])

Grouping the tweets

In [ ]:
modified_data=[]
c = 0
s = ''
for i in range(len(tweets)):
  s = s + tweet_df.loc[i]['content']
  c= c+1
  if c == 10:
    data = [tweet_df.loc[i]['topic'],s]
    modified_data.append(data)
    c = 0
    s = ''

In [ ]:
df = pd.DataFrame(modified_data,columns=["topic","content"])

Pre Processing

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/NLP.csv")

Lower Casing

In [ ]:
df['content'] = df['content'].apply(lambda x: x.lower()) 

Translating to English

In [ ]:
!pip3 install googletrans==3.1.0a0

In [ ]:
from googletrans import Translator

In [ ]:
translator = Translator()
def translate(x):
  text_to_translate = translator.translate(x,dest= 'en')
  return text_to_translate.text

In [ ]:
df['content'] = df['content'].apply(lambda x: translate(x))

Enpansion of Words

In [ ]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how does",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
" u ": " you ",
" ur ": " your ",
" n ": " and "}
def expand(x):
        for key in contractions:
            value = contractions[key]
            x = x.replace(key, value)
        return x;

df['content'] = df['content'].apply(lambda x: expand(x))
df['c_summary'] = df['c_summary'].apply(lambda x: expand(x))

Removing URL

In [ ]:
import re

In [ ]:
df['content'] = df['content'].apply(lambda x: re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', x))
df['c_summary'] = df['c_summary'].apply(lambda x: re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', x))

Removing RT

In [ ]:
df['content'] = df['content'].apply(lambda x: re.sub('RT', "", x))
df['c_summary'] = df['c_summary'].apply(lambda x: re.sub('RT', "", x))


Removing Emojis

In [ ]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [ ]:
df['content'] = df['content'].apply(lambda x: deEmojify(x))
df['c_summary'] = df['c_summary'].apply(lambda x: deEmojify(x))


Removing HTML tags

In [ ]:
from bs4 import BeautifulSoup
df['content'] = df['content'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
df['c_summary'] = df['c_summary'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

Removing Special Characters and Punctuations

In [11]:
punctuation = "!”$%&’()*+-/:;<=>[]^_`{|}~•@#"
def removepunc(x):
    return x.translate(str.maketrans('','',punctuation))

In [12]:
df['content'] = df['content'].apply(lambda x: removepunc(x) )
df['c_summary'] = df['c_summary'].apply(lambda x: removepunc(x) )

In [13]:
def removen(x):
    return x.replace("\n","")
def removes(x):
    return x.replace("&","and")

In [14]:
df['content'] = df['content'].apply(lambda x: removen(x) )
df['content'] = df['content'].apply(lambda x: removes(x) )
df['c_summary'] = df['c_summary'].apply(lambda x: removen(x) )
df['c_summary'] = df['c_summary'].apply(lambda x: removes(x) )

Removal of Accented Chars

In [15]:
import unicodedata
def remove_accented_chars(x):
    x = unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return x

In [16]:
df['content'] = df['content'].apply(lambda x: remove_accented_chars(x))
df['c_summary'] = df['c_summary'].apply(lambda x: remove_accented_chars(x))

Final Dataset

In [17]:
df.to_csv("new_NLP.csv")